In [1]:
import numpy as np
import pandas as pd
import torch

import torch.optim as optim

import torch.nn as nn
from torch.autograd import Variable

In [2]:
np.random.seed(68)  # bureau 68
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.set_default_tensor_type(torch.FloatTensor)

t_c = [1.0]
t_u = [1.0]
t_c = np.array(t_c)
t_u = np.array(t_u)

In [3]:
def numpy_to_variable(r):
    return Variable(torch.from_numpy(r).float(), requires_grad=True).to(device)

In [4]:
model_nn = nn.Sequential(
    nn.Linear(1,2),  # input layer
    nn.ReLU(),
    nn.Linear(2,2),  # hidden layer
    nn.ReLU(), 
    nn.Linear(2,1)   # output layer
).to(device)

In [11]:
n_epochs = 2
learning_rate = 1e-2
optL = optim.SGD(model_nn.parameters(), lr=learning_rate)

loss_fn = torch.nn.MSELoss(size_average = False, reduce = False,reduction='None')

t1_u = numpy_to_variable(t_u[:])
t_ucol = t1_u.clone().detach().requires_grad_(True)
# t_ucol = torch.tensor(t1_u, device=device, requires_grad=True)
t1_c = numpy_to_variable(t_c[:])
t_ccol = t1_c.clone().detach().requires_grad_(True)

In [6]:
for epochs in range(n_epochs):

    t_u1 = torch.cat([t_ucol.view((-1, 1))], dim=1)
    t_c1 = torch.cat([t_ccol.view((-1, 1))], dim=1)

#
#  Extract the random weights and biases from the model
#
    
    # Input Linear Layer
    wt_L0 = model_nn[0].weight.detach().to('cpu').numpy()
    bias_L0 = model_nn[0].bias.detach().to('cpu').numpy()
    
    # Hidden Layer
    wt_L2 = model_nn[2].weight.detach().to('cpu').numpy()
    bias_L2 = model_nn[2].bias.detach().to('cpu').numpy()
    
    # Output Layer
    wt_L4 = model_nn[4].weight.detach().to('cpu').numpy()
    bias_L4 = model_nn[4].bias.detach().to('cpu').numpy()
    

    print(f"epoch:{epochs}, weight_input_layer: {wt_L0}")
    print(f"epoch:{epochs}, bias_input_layer: {bias_L0}")
    print(" ")
    
    print(f"epoch:{epochs}, weight_hidden_layer: {wt_L2}")
    print(f"epoch:{epochs}, bias_hidden_layer: {bias_L2}")
    print(" ")
    
    print(f"epoch:{epochs}, weight_output_layer: {wt_L4}")
    print(f"epoch:{epochs}, bias_output_layer: {bias_L4}")
    print(" ") 
#
#  print output from forward feed
#
    
    t_output = model_nn(t_u1)
    print("output:",t_output)
    
    loss_train = loss_fn(t_output,t_c1)
    print("loss:",loss_train)
    print(" ")
#
#  zero out gradients and compute gradients of loss with respect to
#  weights and biases.
#
    optL.zero_grad()
    loss_train.backward(retain_graph=True)

    gradw_L0 = model_nn[0].weight.grad.detach().to('cpu').numpy()
    gradb_L0 = model_nn[0].bias.grad.detach().to('cpu').numpy()

    gradw_L2 = model_nn[2].weight.grad.detach().to('cpu').numpy()
    gradb_L2 = model_nn[2].bias.grad.detach().to('cpu').numpy()

    gradw_L4 = model_nn[4].weight.grad.detach().to('cpu').numpy()
    gradb_L4 = model_nn[4].bias.grad.detach().to('cpu').numpy()

    print(f"epoch:{epochs}, grad_w_input_layer: {gradw_L0}")
    print(f"epoch:{epochs}, grad_b_input_layer: {gradb_L0}")
    print(" ")
    
    print(f"epoch:{epochs}, grad_w_hidden_layer: {gradw_L2}")
    print(f"epoch:{epochs}, grad_b_hidden_layer: {gradb_L2}")
    print(" ")

    print(f"epoch:{epochs}, gradw_output_layer: {gradw_L4}")
    print(f"epoch:{epochs}, gradb_output_layer: {gradb_L4}")
    print(" ") 

    curr_lr = optL.param_groups[0]["lr"]
    print("learning rate:",curr_lr)

    optL.step()
    
    print("###")

epoch:0, weight_input_layer: [[-0.00748682]
 [ 0.5364436 ]]
epoch:0, bias_input_layer: [-0.82304513 -0.735939  ]
 
epoch:0, weight_hidden_layer: [[-0.27234524  0.1896159 ]
 [-0.01401001  0.56065756]]
epoch:0, bias_hidden_layer: [-0.06275153  0.18710935]
 
epoch:0, weight_output_layer: [[-0.2136969  -0.13899273]]
epoch:0, bias_output_layer: [-0.6755334]
 
output: tensor([[-0.7015]], grad_fn=<AddmmBackward0>)
loss: tensor([[2.8952]], grad_fn=<MseLossBackward0>)
 
epoch:0, grad_w_input_layer: [[0.]
 [0.]]
epoch:0, grad_b_input_layer: [0. 0.]
 
epoch:0, grad_w_hidden_layer: [[0. 0.]
 [0. 0.]]
epoch:0, grad_b_hidden_layer: [0.         0.47300345]
 
epoch:0, gradw_output_layer: [[ 0.        -0.6367482]]
epoch:0, gradb_output_layer: [-3.4030805]
 
learning rate: 0.01
###
epoch:1, weight_input_layer: [[-0.00748682]
 [ 0.5364436 ]]
epoch:1, bias_input_layer: [-0.82304513 -0.735939  ]
 
epoch:1, weight_hidden_layer: [[-0.27234524  0.1896159 ]
 [-0.01401001  0.56065756]]
epoch:1, bias_hidden_laye

### Tensorflow

In [3]:
!pip install --upgrade tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/1b/66/2f47c39cfedb29188d82555d0184a619a0bf8234fd5e5301940efb0aa464/tensorflow-2.13.0-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.13.0 from https://files.pythonhosted.org/packages/2b/ad/d3a2e335004d178e0599cf8aff6c2a92cd21eb9789358fb8f3f951009930/tensorflow_intel-2.13.0-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3 from https://files.pythonhosted.org/packages/99/7b/1085be78abfb2b5f6467ccb316d6bbde59633e8f889c5ba605ebfd0a6ef8/protobuf-4.23.4-cp39-cp39-win_amd64.whl.metadata
     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.6 MB 1.3 MB/s eta 0:00:05
     - -------------------------------------- 0.2/5.6 MB 1.5 MB/s eta 0:00:04
     -- ------------------------------------- 

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\parzi\\anaconda3\\lib\\site-packages\\grpcio-1.42.0.dist-info\\METADATA'



In [7]:
!pip install --upgrade protobuf

  Obtaining dependency information for protobuf from https://files.pythonhosted.org/packages/99/7b/1085be78abfb2b5f6467ccb316d6bbde59633e8f889c5ba605ebfd0a6ef8/protobuf-4.23.4-cp39-cp39-win_amd64.whl.metadata
  Using cached protobuf-4.23.4-cp39-cp39-win_amd64.whl.metadata (540 bytes)
   ---------------------------------------- 0.0/422.5 kB ? eta -:--:--
   -- ------------------------------------- 30.7/422.5 kB ? eta -:--:--
   ------------- -------------------------- 143.4/422.5 kB 1.7 MB/s eta 0:00:01
   ------------------------------- -------- 337.9/422.5 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 422.5/422.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninstalling protobuf-3.19.1:
      Successfully uninstalled protobuf-3.19.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.44.0 requires cloudpickle~=2.2.0, which is not installed.
apache-beam 2.44.0 requires grpcio!=1.48.0,<2,>=1.33.1, which is not installed.
apache-beam 2.44.0 requires regex>=2020.6.8, which is not installed.
apache-beam 2.44.0 requires requests<3.0.0,>=2.24.0, which is not installed.
apache-beam 2.44.0 requires typing-extensions>=3.7.0, which is not installed.
mediapipe 0.9.3.0 requires attrs>=19.1.0, which is not installed.
tensorboard 2.9.1 requires google-auth<3,>=1.6.3, which is not installed.
tensorboard 2.9.1 requires grpcio>=1.24.3, which is not installed.
tensorboard 2.9.1 requires requests<3,>=2.21.0, which is not installed.
tensorflow 2.9.1 requires grpcio<2.0,>=1.24.3, which is not installed.
tensorflow 2.9.1 requires packaging, which is not installed.
tensorflow 2.9.1 requires typing-exten

In [8]:
import tensorflow as tf
from tensorflow import keras

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
fashiondata = tf.keras.datasets.mnist